# Analysis of the historic records of stop and search data from MPS 


In [ ]:
# Located at: link https://data.police.uk/data/

# Consists of ~ 5000 excel files from the MPS database 
# From time range of 2021 to 2024

# data organised in folders of month - year
#each folder contains ~ 127 Excel files
# each folder has 3 files for each force
# 1.name-of-force-outcomes
# 2. name-of-force-stop-and-search
# 3. name-of-force-outcomes - street level crime



: 

In [ ]:
# move the files from individual folders to one big folder for outcomes, stop and search and street level crime

# big folder contains combined three data types

# import os is used for interacting with the operating system 
import os

# import shutil is used for high level operations on files and collections of files
import shutil

""""

def move_excel_files(master_source_dir, destination_directory_stop_and_search):
    for folder in os.listdir(master_source_dir):
        folder_path = os.path.join(master_source_dir, folder)
        if os.path.isdir(folder_path):
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    #checking to see if the file contains Stop and Search data
                    if file.endswith(".xlsx") or file.endswith(".xls") or file.endswith(".csv") :
                        source_file = os.path.join(root, file)
                        destination_file = os.path.join(destination_directory_stop_and_search, file)
                        try:
                            shutil.move(source_file, destination_file)
                            #print(f"Moved: {source_file} to {destination_file}")
                        except Exception as e:
                            print(f"Error moving {source_file}: {e}")

# Specify the master source directory containing the 39 folders
master_source_directory = r'C:\Users\Nathan\Documents\Data Science Projects\Police Dataset\Police Dataset'

# Specify the destination directory where all Excel files will be moved
# This is folder which contains all the stop and search data

destination_directory_stop_and_search = r'C:\Users\\Nathan\Documents\Data Science Projects\Police Dataset\Police Dataset\Stop and Search'

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory_stop_and_search, exist_ok=True)

# Call the function to move Excel files into the master stop and search folder
move_excel_files(master_source_directory, destination_directory_stop_and_search)
print("Done")

%time
%timeit

"""

In [ ]:
# Now have all the data in one place, create dataframes for each data type

In [ ]:
#Import all the neccessry packages

import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import seaborn as sns

import sys #used for 
import csv
import glob # used to access specific path names

print ("it worked")

%time
%timeit

In [ ]:
# get data files names
print ("it worked")

%time
%timeit



# Step 4: Import Excel Files into Pandas ########

In [ ]:
import glob
import pandas as pd

all_excels = glob.glob(r"/Users/nathanstuttard/Documents/Data Science Projects/Police Dataset/Police Dataset/Stop and Search\*.csv")

# get the absolute path of all Excel files 
# all_excel_files = glob.glob(r"C:\Users\Nathan\Documents\Data Science Projects\Police Dataset\Police Dataset\Stop and Search\*.csv")

# Check if any Excel files were found
if not all_excel_files:
    print("No Excel files found.")
else:
    print(f"Fouddddddnd {len(all_excel_files)} Excel files.")

li = []
for filename in all_excel_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# read all Excel files at once
#df = pd.concat(pd.read_excel(excel_file) for excel_file in all_excel_files)

%time
%timeit

In [ ]:
df = pd.concat(li, axis=0, ignore_index=True)

print("dsjndsnjdscjk")

In [ ]:
df